# Customer Profiling

## 1. เรียกใช้ webdriver (chromedriver)

In [1]:
# ใช้ selenium ในการทำงานนี้!

# !pip install selenium
# !pip install pythainlp

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException


import time
import pandas as pd


import pythainlp # ใช้ทำ NlP เกี่ยวกับภาษาไทย พัฒนาโดยคนไทยด้วย!
from pythainlp.corpus.common import thai_stopwords
import nltk
from nltk.corpus import stopwords

In [2]:
# ใช้ chromedriver ในการเข้าไปดึงข้อมูล

# ตั้งค่าให้ Chrome ไม่มี notification มากวน
options = Options()
prefs = {"profile.default_content_setting_values.notifications": 2}
options.add_experimental_option("prefs", prefs)

# เปิด Google Chrome ด้วย chromedriver
chromedriver_path = r'\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(executable_path = chromedriver_path, options=options)


driver.get('https://www.facebook.com') # เข้าไปที่หน้า facebook

C:\Users\pbnat\AppData\Local\Temp\ipykernel_11488\4174588932.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = chromedriver_path, options=options)


## 2. ล็อกอินเฟสบุ๊ค

In [3]:
driver.get('https://www.facebook.com') # เข้าไปที่หน้า facebook

# ระบุตำแหน่งช่องกรอกอีเมล
email_input_elm = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div/div/div/div[2]/div/div[1]/form/div[1]/div[1]/input')

# กรอกอีเมลที่จะใช้ล็อกอิน
email_input_elm.send_keys('email@email.com')

In [4]:
# ระบุตำแหน่งช่องกรอกรหัสผ่าน
pw_input_elm = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div/div/div/div[2]/div/div[1]/form/div[1]/div[2]/div/input')

# กรอกรหัสผ่านที่จะใช้ล็อกอิน
pw_input_elm.send_keys('password')

In [5]:
# ระบุตำแหน่งปุ่ม login
login_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div/div/div/div[2]/div/div[1]/form/div[2]/button')

# login
login_button.click()

## 3. เข้าไปดูการกดไลค์เพจต่างๆ ของ target เรา

In [6]:
target_profile_link = 'https://www.facebook.com/user/likes'

time.sleep(1) # 1 sec delays

driver.get(target_profile_link)

## 4. ดึงรายละเอียดของแต่ละเพจที่กดไลค์

In [7]:
# ระบุตำแหน่ง element ของเพจแรกที่แสดงในหน้า likes
all_liked_pages_xpath = '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div/div/div[1]/div/div/div/div/div[3]/div'

driver.execute_script("document.body.style.zoom= '1%' ") # ซูมออกเพื่อให้ facebook โหลดเพจทั้งหมด

time.sleep(30) # ดีเลย์ 30 วินาที เพื่อรอให้หน้าเพจต่างๆโหลดและแสดงขึ้นมาให้ได้มากที่สุด

driver.execute_script("document.body.style.zoom= '100%' ") # ซูมเข้าเพื่อเตรียมพร้อมในการดึงข้อมูล

all_liked_pages = driver.find_elements(By.XPATH, all_liked_pages_xpath) # หาตำแหน่ง element ของแต่ละเพจที่ target ของเรากดไลค์

In [8]:
def gen_data_dict(all_liked_pages):

    category_dict = dict()
    description_dict = dict()

    for liked_page in all_liked_pages:

        a = ActionChains(driver)
        a.move_to_element(liked_page).perform() # เอาเมาส์ไป hover เพื่อให้โชว์รายละเอียดของเพจ

        count = 0

        while True: # ใส่ while loop เพื่อให้แน่ใจว่าดึงข้อมูลออกมาได้จริงๆ
            try:
                
                # ระบุตำแหน่ง element ของชื่อเพจ และดึงข้อมูลชื่อเพจ
                page_name = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[2]/div/div/div[1]/div[1]/div/div/div[1]/div/div[2]/div/div/div[1]/div/div[1]/div[1]')
                name_text = page_name.text

                if name_text !=  '' : # จะ loop จนกว่าจะไม่มีเพจต่อไปให้ดึงแล้ว
                    
                    # ระบุตำแหน่ง element ประเภทเพจ และดึงข้อมูลประเภทเพจ
                    page_category = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[2]/div/div/div[1]/div[1]/div/div/div[1]/div/div[2]/div/div/div[1]/div/div[1]/div[2]')
                    category_dict[name_text] = page_category.text

                    try:
                        
                        # ระบุตำแหน่ง element คำอธิบายเพจ และดึงข้อมูลคำอธิบายเพจ
                        page_description = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[2]/div/div/div[1]/div[1]/div/div/div[1]/div/div[2]/div/div/div[1]/div/div[2]/span')
                        description_dict[name_text] = page_description.text

                    # บางเพจจะไม่ได้เขียนคำอธิบายเพจเอาไว้ ถ้า selenium หาคำอธิบายเพจไม่เจอให้ใส่ค่าว่าง
                    except NoSuchElementException:
                        description_dict[name_text] = ''

                    break

            except NoSuchElementException:
                pass # หากเกิด error ที่ว่า selenium หาตำแหน่งต่างๆไม่เจอ ให้ข้าม error ไปเพื่อวนลูปใหม่เลย

            count += 1
            if count == 500:
                break # หยุด loop เมื่อ count ครบ 500 ป้องกัน worst case scenario
    
    return category_dict, description_dict

In [9]:
category_dict, description_dict = gen_data_dict(all_liked_pages)

In [10]:
result_df = pd.DataFrame([category_dict, description_dict])
result_df = result_df.transpose()
result_df.columns = ['category', 'description']
result_df.reset_index(inplace=True)

# ตั้งชื่อคอลัมน์เป็น page_name(ชื่อเพจ) , category(ประเภทของเพจ), description(คำอธิบายเพจ)
result_df.columns = ['page_name','category', 'description']

result_df

,page_name,category,description
0,222 Gunpla - Triple Two Gunpla,ร้านศิลปะและหัตถกรรม,จำหน่ายปั๊มลมแอร์บรัช สีและอุปกรณ์สำหรับงานพ่น...
1,4NOLOGUE,บริษัท,"4NOLOGUE Co., Ltd."
2,555,ความสนใจ,
3,A:N,ศิลปิน,ปวดหลัง
4,Ace of Spades [ Thailand ],เกม/ของเล่น,เกมแนว FPS ที่ผสมผสานกับ Minecraft ดูเพิ่มเต
...,...,...,...
276,โคทัตสึ II,นักแปล,แปลงานแฟนอาร์ตและออริจินัลแนวอนิเมะ งานแปลทุกช...
277,โต ติงต๊อง ออดิโอ อีเวนท์ แอนด์ คอมมิวนิเคชั่น,โสตทัศนูปกรณ์,"Microphone & Production RF Specialist, Product..."
278,โปรแกรมแต่งรูป,ชุมชน,โปรแกรมแต่งรูป โหลดโปรแกรมแต่งรูป ht
279,โรงเรียนนวมินทราชินูทิศ หอวัง นนทบุรี,โรงเรียน,


In [11]:
result_df.to_excel('Liked_Pages_Category_Description.xlsx') # เซฟเป็นไฟล์ excel เก็บไว้

## 5. Word Tokenization

In [12]:
## word tokenization

import pythainlp # library เกี่ยวกับการทำ NLP ในภาษาไทย

token_count_dict = dict()

for index, row in result_df.iterrows():
    current_text = row['description']
    current_token = pythainlp.tokenize.word_tokenize(current_text) # แบ่งประโยคเป็นคำๆ
    for token in current_token:
        
        # ถ้าเจอคำที่เคยเจอแล้วให้นับเพิ่ม 1 ครั้ง
        token_count_dict[token] = token_count_dict.get(token, 0) + 1
        
token_count_dict

{'จำหน่าย': 10,
 'ปั๊มลม': 1,
 'แอร์': 1,
 'บ': 1,
 'รัช': 1,
 ' ': 1662,
 'สี': 4,
 'และ': 47,
 'อุปกรณ์': 4,
 'สำหรับ': 8,
 'งาน': 8,
 'พ่น': 1,
 'โมเดล': 12,
 'ศิลปะ': 3,
 'เทวรูป': 1,
 'อื่นๆ': 5,
 'สินค้า': 11,
 'พร้อม': 7,
 'ส่ง': 5,
 'ทุก': 10,
 'รายการ': 2,
 '4': 6,
 'NOLOGUE': 1,
 'Co': 1,
 '.,': 2,
 'Ltd': 1,
 '.': 130,
 'ปวดหลัง': 1,
 'เกม': 23,
 'แนว': 6,
 'FPS': 1,
 'ที่': 31,
 'ผสมผสาน': 1,
 'กับ': 17,
 'Minecraft': 3,
 'ดู': 3,
 'เพิ่ม': 2,
 'เต': 1,
 'Join': 4,
 'us': 7,
 'as': 3,
 'we': 2,
 'unveil': 1,
 'a': 12,
 'more': 4,
 'balanced': 1,
 'way': 2,
 'of': 37,
 'life': 3,
 'RSVP': 1,
 'for': 17,
 '#': 4,
 'NextAtAcer': 1,
 'on': 9,
 'January': 1,
 ',': 76,
 '2023': 1,
 'here': 5,
 ':': 45,
 'https': 12,
 '://': 25,
 'acer': 1,
 'co': 2,
 '/': 44,
 'next-at-acer-fblive': 1,
 'The': 14,
 'official': 11,
 'Facebook': 6,
 'page': 10,
 'Agar': 1,
 'io': 1,
 '!': 28,
 '🦠': 1,
 'Play': 5,
 'online': 3,
 'with': 7,
 'players': 1,
 'around': 3,
 'the': 35,
 'world': 3,
 'you'

In [19]:
# ทำให้อยู่ในรูป dataframe เพื่อให้จัดการได้ง่ายขึ้น

token_df = pd.DataFrame([token_count_dict]).transpose()
token_df.reset_index(inplace=True)
token_df.columns = ['word','count']

# ดูตัวอย่าง dataframe 10 แถวแรก
token_df.head(10)

,word,count
0,จำหน่าย,10
1,ปั๊มลม,1
2,แอร์,1
3,บ,1
4,รัช,1
5,,1662
6,สี,4
7,และ,47
8,อุปกรณ์,4
9,สำหรับ,8


In [20]:
# เก็บเฉพาะข้อมูลที่เป็นตัวหนังสือ

regex = r'[ก-๙a-zA-Z]'

token_df = token_df[ token_df['word'].str.contains(regex) ]

token_df = token_df.sort_values('count', ascending=False) #เรียงให้คำที่เจอบ่อยมากที่สุดมาอยู่ด้านบน

token_df.head(10)

,word,count
7,และ,47
47,of,37
78,the,35
108,com,32
31,ที่,31
125,เพจ,30
182,and,29
28,เกม,23
111,ของ,22
340,กัน,18


In [22]:
# กำจัด stop words

th_stop_words = list(thai_stopwords()) # stop words ภาษาไทย
en_stop_words = list(stopwords.words('english')) # stop words ภาษาอังกฤษ
add_on_stop_words = ['www', 'เพจ', 'The', 'http', 'https', 'com'] # คำอื่นๆที่อาจไม่มีความหมายมากนัก

all_stop_words = th_stop_words + en_stop_words + add_on_stop_words

token_df = token_df[ ~token_df['word'].isin(all_stop_words) ]

token_df.head(10)

,word,count
28,เกม,23
123,Thailand,13
102,ติดต่อ,12
12,โมเดล,12
66,official,11
565,ไทย,11
16,สินค้า,11
131,ข่าวสาร,11
68,page,10
0,จำหน่าย,10


In [16]:
# export เป็นไฟล์ excel เพื่อใช้วิเคราะห์ต่อไป
token_df.to_excel('token_word_count.xlsx')